In [1]:
from collections import deque
import tensorflow as tf
import numpy as np
import kidney
import tensorflow.contrib.layers as skflow
import matplotlib.pyplot as plt
import pandas as pd
import random
%matplotlib inline

In [2]:
ENV_NAME = 'Kidney'  # Environment name
NUM_PATIENTS = 20
NUM_EPISODES = 100000  # Number of episodes the agent plays
GAMMA = .99  # Discount factor
LAMBDA = .9  # Regularization factor
KEEP_PROB = .8  # Dropout keep prob rate
EXPLORATION_STEPS = 1000000 # Number of steps over which the initial value of epsilon is linearly annealed to its final value
INITIAL_EPSILON = 0.2  # Initial value of epsilon in epsilon-greedy
FINAL_EPSILON = 0.000  # Final value of epsilon in epsilon-greedy
INITIAL_REPLAY_SIZE = 256  # Number of steps to populate the replay memory before training starts
NUM_REPLAY_MEMORY = 2000  # Number of replay memory the agent uses for training
BATCH_SIZE = 32  # Mini batch size
TARGET_UPDATE_INTERVAL = 250  # The frequency with which the target network is updated
TEST_EVERY = 100
TRAIN_INTERVAL = 4  # The agent selects 4 actions between successive updates
LEARNING_RATE = 0.025  # Learning rate used by RMSProp
MOMENTUM = 0.95  # Momentum used by RMSProp
MIN_GRAD = 0.01  # Constant added to the squared gradient in the denominator of the RMSProp update
NO_OP_STEPS = 0  # Maximum number of "do nothing" actions to be performed by the agent at the start of an episode
LOAD_NETWORK = False
TRAIN = True
SAVE_NETWORK_PATH = 'saved_networks/' + ENV_NAME
SAVE_SUMMARY_PATH = 'summary/' + ENV_NAME
NUM_EPISODES_AT_TEST = 1  # Number of episodes the agent plays at test time
PRINT_EVERY = 100
PLOT_EVERY = 250

In [3]:
env = kidney.KidneyExchange(init_size = NUM_PATIENTS)

In [4]:
num_actions = 45
state_dim = 16
discount_factor = 1

In [5]:
class Agent():
    def __init__(self, state_dim, num_actions):
        self.state_dim = state_dim
        self.num_actions = num_actions
        self.epsilon = INITIAL_EPSILON
        self.epsilon_step = (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORATION_STEPS
        self.t = 0

        # Parameters used for summary
        self.total_reward = 0
        self.total_q_max = 0
        self.total_loss = 0
        self.duration = 0
        self.episode = 0
        
        self.training_loss = []

        # Create replay memory
        self.replay_memory = deque(maxlen = NUM_REPLAY_MEMORY)

        # Create q network
        with tf.variable_scope("q_network"):
            self.s, self.q_values = self.build_network()
        self.q_network_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="q_network")
        
        # Create target network
        with tf.variable_scope("target_network"):
            self.st, self.target_q_values = self.build_network()
        self.target_network_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="target_network")
        
        
        self.a, self.y, self.loss, self.grad_update = self.build_training_op(self.q_network_weights)
        
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.initialize_all_variables())


   
    def build_network(self):
        s = tf.placeholder(tf.float32, (None, self.state_dim), name="states")
        net = skflow.fully_connected(s, 5,
                                    weights_initializer = skflow.xavier_initializer(uniform = False),
                                    weights_regularizer = skflow.l2_regularizer(LAMBDA))
        net = skflow.dropout(net, KEEP_PROB)
        net = skflow.fully_connected(net, 5,
                                    weights_initializer = skflow.xavier_initializer(uniform = False),
                                    weights_regularizer = skflow.l2_regularizer(LAMBDA))
        net = skflow.dropout(net, KEEP_PROB)
        q_values = skflow.fully_connected(net, self.num_actions,
                                    weights_initializer = skflow.xavier_initializer(uniform = False),
                                    weights_regularizer = skflow.l2_regularizer(LAMBDA))
        return s, q_values
    
    def get_action(self, state, avail, train = True):
        if train and np.random.uniform() <= self.epsilon:
            avail_actions = np.arange(self.num_actions)[avail.flatten()]
            #import pdb; pdb.set_trace()
            action = np.random.choice(avail_actions)
        else:
            qvals = self.q_values.eval(feed_dict={self.s: state})
            qvals[~avail] = -np.inf 
            action = np.argmax(qvals)
            
        if train:
            # Anneal epsilon linearly over time
            if self.epsilon > FINAL_EPSILON and self.t >= INITIAL_REPLAY_SIZE:
                self.epsilon -= self.epsilon_step
        return action

    
    def run(self, state, action, reward, next_state, terminal):
        
        # Store transition in replay memory
        self.replay_memory.append((state, action, reward, next_state, terminal))
       
        if self.t >= INITIAL_REPLAY_SIZE:
            # Train network
            if self.t % TRAIN_INTERVAL == 0:
                loss = self.train_network()
                self.training_loss.append(loss)
                
            # Update target network
            if self.t % TARGET_UPDATE_INTERVAL == 0:
                self.update_target_network()
            
        self.t += 1
        

    def update_target_network(self):
        for v_q, v_target in zip(self.q_network_weights, self.target_network_weights):
            v_target.assign(v_q)
    
    
    def train_network(self):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        terminal_batch = []
        y_batch = []

        # Sample random minibatch of transition from replay memory
        minibatch = random.sample(self.replay_memory, BATCH_SIZE)
        for data in minibatch:
            state_batch.append(data[0])
            action_batch.append(data[1])
            reward_batch.append(data[2])
            next_state_batch.append(data[3])
            terminal_batch.append(data[4])

        # Convert to appropriate types
        state_batch = np.vstack(state_batch).astype(np.float32)
        reward_batch = np.vstack(reward_batch).astype(np.float32)
        action_batch = np.vstack(action_batch).astype(np.float32)
        next_state_batch = np.vstack(next_state_batch).astype(np.float32)
        terminal_batch = np.vstack(terminal_batch).astype(np.int32)
        
        # Find max_a Qhat(s[t+1],a)
        target_q_values_batch = self.target_q_values.eval(feed_dict={self.st: next_state_batch})
        y_batch = reward_batch + (1 - terminal_batch) * GAMMA * np.max(target_q_values_batch, axis=1).reshape(-1, 1)

        # Compute loss and update
        loss, _ = self.sess.run([self.loss, self.grad_update], feed_dict={
            self.s: state_batch,
            self.a: action_batch,
            self.y: y_batch
        })
        return loss

        
    def build_training_op(self, q_network_weights):
        a = tf.placeholder(tf.int64, [None,1], name = "a")
        y = tf.placeholder(tf.float32, [None,1], name = "y")

        # Convert action to one hot vector
        a_one_hot = tf.one_hot(a, self.num_actions, 1.0, 0.0)
        q_value = tf.reduce_sum(tf.mul(self.q_values, a_one_hot), reduction_indices=1)

        # Clip the error, the loss is quadratic when the error is in (-1, 1), and linear outside of that region
        error = tf.abs(y - q_value)
        quadratic_part = tf.clip_by_value(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = tf.reduce_mean(0.5 * tf.square(quadratic_part) + linear_part)

        #optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE, momentum=MOMENTUM, epsilon=MIN_GRAD)
        optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
        grad_update = optimizer.minimize(loss, var_list=q_network_weights)

        return a, y, loss, grad_update



In [6]:
env = kidney.KidneyExchange(init_size = NUM_PATIENTS)
agent = Agent(num_actions = 45, state_dim = 16)
losses = deque(maxlen = 100)
match_prob = []

In [7]:
fixed_state = env.reset(True).flatten()

In [15]:
for i_episode in range(NUM_EPISODES):
    terminal = False
    state = env.reset(True)
    avail = env.available_actions()
    max_patients = env.get_max_patients()
    if max_patients == 0: 
        i_episode -= 1
        continue
    matched_patients = 0
    while not terminal:
        action = agent.get_action(state, avail)
        next_state, reward, terminal, avail = env.step(action, False)
        matched_patients += 2
        if terminal: 
            penalty = matched_patients - max_patients
            reward += penalty 
        agent.run(state, action, reward, next_state, terminal)
        
    losses.append(agent.training_loss)
    if i_episode % PRINT_EVERY == 0 and len(losses) > 20:
        print(i_episode, np.mean(losses), agent.epsilon)
        
    if i_episode % PLOT_EVERY == 0:
        fig, ax = plt.subplots(1, 2, figsize = (15, 3))
        ax[0].plot(match_prob)
        if len(match_prob) > 100:
            pd.Series(match_prob).rolling(100).mean().plot(ax = ax[1])
        fig.savefig("match_prob")
        plt.close("all")
        
    if i_episode % TEST_EVERY == 0:
        terminal = False
        state = env.reset(True)
        avail = env.available_actions()
        max_patients = env.get_max_patients()
        if max_patients == 0: continue
        matched_patients = 0
        while not terminal:
            action = agent.get_action(state, avail)
            next_state, reward, terminal, avail = env.step(action, False)
            matched_patients += 2
            if terminal: 
                penalty = matched_patients - max_patients
                reward += penalty 

        print("Matched {} out of {} possible".format(matched_patients, max_patients))
        match_prob.append(matched_patients / max_patients)

0 0.694767 -3.691236499641057e-12
Matched 18 out of 20 possible
100 0.694757 -3.691236499641057e-12
Matched 18 out of 20 possible
200 0.69474 -3.691236499641057e-12
Matched 8 out of 8 possible
300 0.694725 -3.691236499641057e-12
Matched 18 out of 20 possible
400 0.694715 -3.691236499641057e-12
Matched 18 out of 18 possible
500 0.694704 -3.691236499641057e-12
Matched 18 out of 20 possible
600 0.6947 -3.691236499641057e-12
Matched 12 out of 14 possible
700 0.694686 -3.691236499641057e-12
Matched 18 out of 18 possible
800 0.694672 -3.691236499641057e-12
Matched 20 out of 24 possible
900 0.694668 -3.691236499641057e-12
Matched 14 out of 14 possible
1000 0.69466 -3.691236499641057e-12
Matched 16 out of 18 possible
1100 0.69465 -3.691236499641057e-12
Matched 14 out of 14 possible
1200 0.694632 -3.691236499641057e-12
Matched 14 out of 16 possible
1300 0.694617 -3.691236499641057e-12
Matched 18 out of 18 possible
1400 0.694604 -3.691236499641057e-12
Matched 18 out of 18 possible
1500 0.694597 

KeyboardInterrupt: 

In [9]:
agent.epsilon

0.04612419999614337

In [10]:
terminal = False
state = env.reset(True)
print(env.state)
avail = env.available_actions()
max_patients = env.get_max_patients()
matched_patients = 0
while not terminal:
    action = agent.get_action(state, avail)
    p1, p2 = env.exchanges[action]
    print(env.how_demanded(p1), env.how_demanded(p2))
    next_state, reward, terminal, avail = env.step(action, False)
    matched_patients += 2
    if terminal: 
        penalty = matched_patients - max_patients
        reward += penalty 
    agent.run(state, action, reward, next_state, terminal)

(a, a)      4
(a, b)      3
(a, ab)     1
(a, o)      1
(b, a)      2
(b, b)      4
(b, ab)     1
(b, o)      0
(ab, a)     1
(ab, b)     0
(ab, ab)    0
(ab, o)     0
(o, a)      0
(o, b)      1
(o, ab)     2
(o, o)      0
dtype: int64
self_demanded self_demanded
self_demanded self_demanded
recip_demanded recip_demanded
recip_demanded recip_demanded
recip_demanded over_demanded
self_demanded self_demanded
self_demanded self_demanded


In [11]:
max_patients, matched_patients

(16, 14)